In [ ]:
!pip install pytorch_lightning==1.7.0
!pip install quaterion

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.9/700.9 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 14.1 MB/s eta 0:00

## Config

In [ ]:
# List of default parameters, used across multiple training scripts

TRAIN_BATCH_SIZE = 32

# Rescale images to this size
IMAGE_SIZE = 336

## Data preparation

In [ ]:
import os
import pickle
from typing import Callable

import numpy as np
import tqdm
from pytorch_lightning import seed_everything
from torch.utils.data import Dataset, Subset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from quaterion.dataset import GroupSimilarityDataLoader, SimilarityGroupSample

# set seed to deterministically sample train and test categories later on
seed_everything(seed=42)

# dataset will be downloaded to this directory under local directory
dataset_path = os.path.join(".", "torchvision", "datasets")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
import zipfile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# zip extraction
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Dataset_ML/vegetables_datasets.zip', 'r')
zip_ref.extractall('')
zip_ref.close()

In [ ]:
TRAIN_PATH = '/content/vegetables_datasets/train'
TEST_PATH = '/content/vegetables_datasets/test'
VAL_PATH = '/content/vegetables_datasets/validation'

In [ ]:
def get_raw_dataset(input_size: int, split_cache_path="split_cache.pkl"):
    """
    Create dataset for extracting images, associated with vectors.
    Args:
        input_size: Resize images to this size
        split_cache_path: Path to train split

    Returns:

    """
    transform = transforms.Compose(
        [
            transforms.Resize(input_size, max_size=input_size + 1),
        ]
    )

    train_ds = datasets.ImageFolder(TRAIN_PATH, transform=transform)
    test_ds = datasets.ImageFolder(TEST_PATH, transform=transform)
    full_ds = train_ds + test_ds

    # Use same indexes, as was used for training
    train_indices, test_indices = pickle.load(open(split_cache_path, "rb"))

    train_dataset = Subset(full_ds, train_indices)

    test_dataset = Subset(full_ds, test_indices)

    return train_dataset, test_dataset

In [ ]:
def get_datasets(
    input_size: int,
    split_cache_path="split_cache.pkl",
):
    # Use Mean and std values for the ImageNet dataset as the base model was pretrained on it.
    # taken from https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # create train and test transforms
    train_transform = transforms.Compose(
        [
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ]
    )

    test_transform = transforms.Compose(
        [
            transforms.Resize((input_size, input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ]
    )

    # we need to merge train and test splits into a full dataset first,
    # and then we will split it to two subsets again with each one composed of distinct labels.

    train_ds = datasets.ImageFolder(TRAIN_PATH)
    test_ds = datasets.ImageFolder(TEST_PATH)
    full_ds = train_ds + test_ds

    train_indices, test_indices = None, None

    if not split_cache_path or not os.path.exists(split_cache_path):
        # full_dataset contains examples from 196 categories labeled with an integer from 0 to 195
        # randomly sample half of it to be used for training
        train_categories = np.random.choice(a=196, size=196 // 2, replace=False)

        # get a list of labels for all samples in the dataset
        labels_list = np.array([label for _, label in tqdm.tqdm(full_ds)])

        # get a mask for indices where label is included in train_categories
        labels_mask = np.isin(labels_list, train_categories)

        # get a list of indices to be used as train samples
        train_indices = np.argwhere(labels_mask).squeeze()
        # print(train_indices)

        # others will be used as test samples
        test_indices = np.argwhere(np.logical_not(labels_mask)).squeeze()

    if train_indices is None or test_indices is None:
        train_indices, test_indices = pickle.load(open(split_cache_path, "rb"))
    else:
        pickle.dump((train_indices, test_indices), open(split_cache_path, "wb"))

    # now that we have distinct indices for train and test sets, we can use `Subset` to create new datasets
    # from `full_dataset`, which contain only the samples at given indices.
    # finally, we apply transformations created above.

    train_dataset = VegDataset(
        Subset(full_ds, train_indices), transform=train_transform
    )

    test_dataset = VegDataset(
        Subset(full_ds, test_indices), transform=test_transform
    )

    return train_dataset, test_dataset

In [ ]:
def get_dataloaders(
    batch_size: int,
    input_size: int,
    shuffle: bool = False,
    split_cache_path="split_cache.pkl",
):
    train_dataset, test_dataset = get_datasets(input_size, split_cache_path)
    print(len(train_dataset))
    train_dataloader = GroupSimilarityDataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle
    )

    test_dataloader = GroupSimilarityDataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    return train_dataloader, test_dataloader

In [ ]:
class VegDataset(Dataset):
    def __init__(self, dataset: Dataset, transform: Callable):
        self._dataset = dataset
        self._transform = transform

    def __len__(self) -> int:
        return len(self._dataset)

    def __getitem__(self, index) -> SimilarityGroupSample:
        image, label = self._dataset[index]
        image = self._transform(image)

        return SimilarityGroupSample(obj=image, group=label)

## Feature Vector Encoder

In [ ]:
from typing import Dict, Optional, Union

import torch
from quaterion_models.encoders import Encoder
from quaterion_models.heads import EncoderHead, SkipConnectionHead
from torch import nn

from quaterion import TrainableModel
from quaterion.eval.attached_metric import AttachedMetric
from quaterion.eval.group import RetrievalRPrecision
from quaterion.loss import SimilarityLoss, TripletLoss
from quaterion.train.cache import CacheConfig, CacheType
import torchvision

In [ ]:
import torch
import torch.nn as nn
from quaterion_models.encoders import Encoder


class VegEncoder(Encoder):
    def __init__(self, encoder_model: nn.Module):
        super().__init__()
        self._encoder = encoder_model
        self._embedding_size = 2048  # last dimension from the ResNet model

    @property
    def trainable(self) -> bool:
        return False

    @property
    def embedding_size(self) -> int:
        return self._embedding_size

    def forward(self, images):
        embeddings = self._encoder.forward(images)
        return embeddings

    def save(self, output_path: str):
        os.makedirs(output_path, exist_ok=True)
        torch.save(self._encoder, os.path.join(output_path, "encoder.pth"))

    @classmethod
    def load(cls, input_path):
        encoder_model = torch.load(os.path.join(input_path, "encoder.pth"))
        return VegEncoder(encoder_model)

In [ ]:
class Model(TrainableModel):
    def __init__(self, lr: float, mining: str):
        self._lr = lr
        self._mining = mining
        super().__init__()

    def configure_encoders(self) -> Union[Encoder, Dict[str, Encoder]]:
        pre_trained_encoder = torchvision.models.resnet152(weights='IMAGENET1K_V1')
        pre_trained_encoder.fc = nn.Identity()
        return VegEncoder(pre_trained_encoder)

    def configure_head(self, input_embedding_size) -> EncoderHead:
        return SkipConnectionHead(input_embedding_size, dropout=0.1)

    def configure_loss(self) -> SimilarityLoss:
        return TripletLoss(mining=self._mining, margin=0.5)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), self._lr)
        return optimizer

    def configure_caches(self) -> Optional[CacheConfig]:
        return CacheConfig(
            cache_type=CacheType.AUTO, save_dir="./cache_dir", batch_size=32
        )

    def configure_metrics(self) -> AttachedMetric:
        return AttachedMetric(
            "rrp",
            metric=RetrievalRPrecision(),
            prog_bar=True,
            on_epoch=True,
            on_step=False,
        )

In [ ]:
import shutil
from quaterion import Quaterion
from pytorch_lightning.callbacks import EarlyStopping, ModelSummary

def train(
    lr: float,
    mining: str,
    batch_size: int,
    epochs: int,
    input_size: int,
    shuffle: bool,
    save_dir: str,
):

    model = Model(
        lr=lr,
        mining=mining,
    )
    import warnings

    warnings.filterwarnings("ignore", ".*does not have many workers.*")

    train_dataloader, val_dataloader = get_dataloaders(
        batch_size=batch_size, input_size=input_size, shuffle=shuffle
    )

    print(train_dataloader)

    Quaterion.fit(
        trainable_model=model,
        trainer=None,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
    )

    shutil.rmtree(save_dir, ignore_errors=True)
    model.save_servable(save_dir)

In [ ]:
train(lr=1e-3, mining="hard", batch_size=32, epochs=30, input_size=IMAGE_SIZE, shuffle=True, save_dir="vegs_class")

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

9600


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/base.py:22: LightningDeprecationWarning: pytorch_lightning.callbacks.base.Callback has been deprecated in v1.7 and will be removed in v1.9. Use the equivalent class from the pytorch_lightning.callbacks.callback.Callback class instead.
  rank_zero_deprecation(
2023-03-09 04:02:10.636 | DEBUG    | quaterion.train.cache_mixin:_cache:168 - Using full cache
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

...

Epoch 1    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 551/563 0:00:14 • 0:00:01 37.22it/s loss: 0.065 v_num: 0              
                                                                                 rrp_validation: 0.998 rrp_train:  
                                                                                 0.914                             
Validation ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 251/263 0:00:07 • 0:00:01 31.80it/s

Epoch 2    ━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 180/563 0:00:04 • 0:00:09 44.68it/s loss: 0.0381 v_num: 0             
                                                                                 rrp_validation: 0.998 rrp_train:  
                                                                                 0.985                             

Epoch 3    ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 238/563 0:00:05 • 0:00:08 42.25it/s loss: 0.027 v_num: 0              
                                                                                 rrp_validation: 0.998 rrp_train:  
                                                                                 0.996                             

Epoch 3    ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━ 460/563 0:00:11 • 0:00:03 40.19it/s loss: 0.0271 v_num: 0             
                                                                                 rrp_validation: 0.998 rrp_train:  
                                                                                 0.996                             
Validation ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━ 160/263 0:00:04 • 0:00:03 36.04it/s

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:1972: LightningDeprecationWarning: `Trainer.reset_train_val_dataloaders` has been deprecated in v1.7 and will be removed in v1.9. Use `Trainer.reset_{train,val}_dataloader` instead
  rank_zero_deprecation(
